In [1]:
import pandas as pd
import numpy as np
import scipy
import sklearn

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel, SelectPercentile, chi2, VarianceThreshold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import metrics

In [4]:
import optuna
from optuna.visualization import plot_intermediate_values

In [5]:
import numpy as np
import xgboost as xgb
import lightgbm as lgb

In [6]:
SEED = 85

In [7]:
from hypster_xgboost import *
from optuna_utils import *

# Get Dataset

In [8]:
from scipy.sparse import csr_matrix, save_npz, load_npz

In [9]:
X_train = load_npz("X_train.npz")
y_train = pd.read_pickle("y_train.pkl")

In [10]:
X_test = load_npz("X_test.npz")
y_test = pd.read_pickle("y_test.pkl")

In [11]:
#pipeline - pipeline_objective OR regular pipeline
#consider making pre-made steps with best practices (FS, scaling, etc...) then add option to concat to make one pipeline 

In [12]:
pipeline = Pipeline([("sel", SelectPercentile(chi2))])
pipe_params = {"sel__percentile" : optuna.distributions.IntUniformDistribution(1,100)}
#pipeline = None
#pipe_params = None

In [13]:
#TODO: automatic seed
xgb_linear = XGBClassifierOptuna(booster_list=['gblinear'])
xgb_tree = XGBClassifierOptuna(booster_list=['gbtree', 'dart'])
#lgb_estimator = LGBClassifierOptuna()
#sgd_estimator = SGDClassifierOptuna()
#rf_estimator  = RFClassifierOptuna()

In [14]:
estimators = [xgb_linear]#, xgb_tree]#, sgd|_estimator]

In [15]:
#TODO
#studies - consider changing the name
#strings for estimators + add "all"
# pipeline - another approach

In [16]:
clf = HyPSTERClassifier(estimators, pipeline, pipe_params,
                        scoring=sklearn.metrics.roc_auc_score, greater_is_better=True,
                        cv=StratifiedKFold(n_splits=3, random_state=SEED), refit=True, random_state=SEED, n_jobs=-1)

In [17]:
%%time
clf.fit(X_train, y_train, n_trials_per_estimator=10)

XGBoost Classifier
Score: 0.5
Score: 0.5
Score: 0.5
Score: 0.5
Score: 0.77166
Score: 0.96835
Score: 0.93221
Score: 0.95435
Wall time: 11 s


In [18]:
final_pipeline = clf.best_estimator_

In [19]:
clf.best_score_

0.9683466171485335

In [20]:
clf.best_params_

{'sel__percentile': 93,
 'init_eta': 0.10820642061534623,
 'booster': 'gblinear',
 'lambda': 2.48705402458801e-09,
 'alpha': 2.5336397727184858e-05,
 'shotgun_feature_selector': 'shuffle'}

In [26]:
final_pipeline = final_pipeline.fit(X_train, y_train)

In [27]:
test_probs = final_pipeline.predict_proba(X_test)
test_probs = test_probs[:,1]

In [28]:
sklearn.metrics.roc_auc_score(y_test, test_probs)

0.9680561826832007

In [37]:
study.best_trial.value

0.9673658171647715